# Tabular Playground Series - Feb 2022

## Importación de librerías a utilizar

In [18]:
import zipfile
import os

## Generación de DataFrame
### Primero descargo la información directo desde la API de Kaggle

In [1]:
conda install -c conda-forge kaggle

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [12]:
!kaggle competitions files -c tabular-playground-series-feb-2022

name                    size  creationDate         
---------------------  -----  -------------------  
sample_submission.csv    3MB  2022-01-31 13:54:40  
test.csv               592MB  2022-01-31 13:54:40  
train.csv                1GB  2022-01-31 13:54:40  


In [20]:
!kaggle competitions download -c tabular-playground-series-feb-2022


  0%|          | 0.00/267M [00:00<?, ?B/s]
  0%|          | 1.00M/267M [00:00<03:02, 1.53MB/s]
  1%|          | 2.00M/267M [00:01<02:08, 2.16MB/s]
  1%|1         | 3.00M/267M [00:01<01:41, 2.72MB/s]
  1%|1         | 4.00M/267M [00:01<01:34, 2.93MB/s]
  2%|1         | 5.00M/267M [00:01<01:22, 3.31MB/s]
  2%|2         | 6.00M/267M [00:02<01:17, 3.55MB/s]
  3%|2         | 7.00M/267M [00:02<01:09, 3.90MB/s]
  3%|2         | 8.00M/267M [00:02<01:05, 4.14MB/s]
  3%|3         | 9.00M/267M [00:02<01:17, 3.47MB/s]
  4%|3         | 10.0M/267M [00:03<01:25, 3.16MB/s]
  4%|4         | 11.0M/267M [00:03<01:17, 3.47MB/s]
  4%|4         | 12.0M/267M [00:04<01:28, 3.01MB/s]
  5%|4         | 13.0M/267M [00:04<01:35, 2.78MB/s]
  5%|5         | 14.0M/267M [00:04<01:25, 3.10MB/s]
  6%|5         | 15.0M/267M [00:05<01:24, 3.12MB/s]
  6%|5         | 16.0M/267M [00:05<01:17, 3.41MB/s]
  6%|6         | 17.0M/267M [00:05<01:14, 3.50MB/s]
  7%|6         | 18.0M/267M [00:06<01:32, 2.84MB/s]
  7%|7         | 19.

### Luego, la descomprimo, borro el zip y genero el DataFrame

In [22]:
with zipfile.ZipFile("tabular-playground-series-feb-2022.zip", "r") as zip_ref:
    zip_ref.extractall(r"C:\Users\alanp\Data Science\kaggle_tabular_playground\datasets")
os.remove("tabular-playground-series-feb-2022.zip")